# 1. Enunciado
Los archivos places_details.csv y places_reviews.csv contienen información sobre reseñas que
han dejado diversos usuarios a múltiples restaurantes. Para este ejercicio, solicitamos que
resumas los principales temas que mencionan los comensales al momento de dejar una reseña
(Hint: Sabor, variedad, servicio, etc).


# 2. Solución

## 2.1. Configuración e importe de paquetes

In [1]:
!pip install gensim nltk
!pip install googletrans==4.0.0-rc1
!pip install --upgrade nbformat
!pip install bertopic

In [2]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from gensim import models, corpora
nltk.download('stopwords')
import os
import pandas as pd
import unicodedata

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\leidy\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


A continuación se definen las constantes del código, en este caso la ubicación de los datos

In [3]:
DATA_PATH = os.path.join('data')

In [4]:
def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    
    # Tokenize
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(text)
    
    # Remove stopwords
    stop_words = set(stopwords.words('spanish'))
    tokens = [token for token in tokens if token not in stop_words]
    
    return tokens

from googletrans import Translator, LANGUAGES

def translate_to_spanish(text, source_language='en'):
    try:
        translator = Translator()
        translated = translator.translate(text, src=source_language, dest='es')
        return translated.text
    except:
        return ''

def remove_accents(input_str):
    nfkd_form = unicodedata.normalize('NFKD', input_str)
    return ''.join([c for c in nfkd_form if not unicodedata.combining(c)])


## 2.2. Carga de datos
Se cuenta con 9183 registros, de los cuales 317 son duplicados y se eliminan quedando con 8866 registros, descritos con 33 variables. Se observa una muestra de 5 registros

In [5]:
detalles = pd.read_csv(os.path.join(DATA_PATH, 'places_details (3) (1).csv'))
reviews = pd.read_csv(os.path.join(DATA_PATH, 'places_reviews (3) (1) (2).csv'))
datos = pd.merge(detalles, reviews, on=['place_id'], how='inner')
print('Número de registros duplicados: ', datos.duplicated().sum())
datos.drop_duplicates(inplace=True)
print('Número de filas y columnas:', datos.shape)
display(datos.language.value_counts())
# Aplicar la función a la columna 'Texto'
datos['text'] = datos['text'].apply(remove_accents)

datos_es = datos[datos['language'].isin(['es'])]
datos_en = datos[datos['language']=='en']

datos_en['text'] = datos_en['text'].apply(translate_to_spanish)
datos_en = datos_en[datos_en['text']!='']
datos_new = pd.concat([datos_en, datos_es])

textos = datos_new['text'].apply(preprocess_text)

textos

Número de registros duplicados:  317
Número de filas y columnas: (8866, 33)


language
es         8371
en          424
pt           23
sl           11
fr            6
id            2
sv            2
ko            2
ar            2
gl            2
ja            2
de            2
zh            2
pl            2
hi            2
zh-Hant       2
it            2
fi            1
tr            1
ro            1
ca            1
ru            1
ku            1
cs            1
Name: count, dtype: int64

C:\Users\leidy\AppData\Local\Temp\ipykernel_8568\4072016196.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datos_en['text'] = datos_en['text'].apply(translate_to_spanish)


33      [bonito, restaurante, buena, comida, instalaci...
81                                                     []
156                                              [fresco]
217     [comida, realmente, sabrosa, restaurante, vist...
230      [gran, servicio, celebramos, cumpleanos, genial]
                              ...                        
9178    [good, place, to, eat, traditional, food, from...
9179    [meseros, siempre, mala, cara, tortas, ahogada...
9180                            [ricas, tortas, ahogadas]
9181    [riquisimo, tortas, ahogadas, tacos, dorados, ...
9182    [perfeccion, existe, doy, 4, estrellas, delici...
Name: text, Length: 8790, dtype: object

In [6]:
preprocessed_reviews = textos.tolist() # Your list of reviews

In [7]:
# Create a dictionary
dictionary = corpora.Dictionary(preprocessed_reviews)

# Create a corpus
corpus = [dictionary.doc2bow(review) for review in preprocessed_reviews]

In [8]:
# Train the LDA model
lda_model = models.LdaModel(corpus, id2word=dictionary, num_topics=5, passes=15)

# Print the topics
for idx, topic in lda_model.print_topics(-1):
    print(f"Topic {idx}: {topic}")

Topic 0: 0.065*"rico" + 0.028*"super" + 0.026*"tacos" + 0.025*"buenos" + 0.019*"ricos" + 0.015*"delicioso" + 0.013*"bien" + 0.013*"servicio" + 0.011*"siempre" + 0.011*"exelente"
Topic 1: 0.023*"bien" + 0.015*"si" + 0.015*"mas" + 0.012*"buenas" + 0.012*"bueno" + 0.012*"solo" + 0.010*"tortas" + 0.009*"tacos" + 0.008*"sabor" + 0.008*"estan"
Topic 2: 0.088*"lugar" + 0.049*"buen" + 0.034*"excelente" + 0.028*"buena" + 0.028*"atencion" + 0.025*"comida" + 0.019*"comer" + 0.019*"sabor" + 0.016*"servicio" + 0.015*"agradable"
Topic 3: 0.073*"comida" + 0.060*"excelente" + 0.046*"servicio" + 0.019*"buena" + 0.017*"rica" + 0.017*"atencion" + 0.016*"deliciosa" + 0.016*"mejor" + 0.014*"precio" + 0.011*"buen"
Topic 4: 0.028*"delicioso" + 0.025*"mas" + 0.010*"experiencia" + 0.010*"menu" + 0.009*"si" + 0.008*"gusto" + 0.006*"restaurante" + 0.005*"cada" + 0.005*"mejor" + 0.004*"tiempo"


## 2.3. Embbedings

In [ ]:
from bertopic import BERTopic

In [20]:
textos = datos['text'].apply(preprocess_text)
textos = [' '.join(texto) for texto in textos if len(texto)>0]
textos

['bonito restaurante buena comida instalaciones limpias personal amable acuerdo comida lleva mas tiempo terrible',
 'fresco',
 'comida realmente sabrosa restaurante vista increible recomiendo sentarme afuera',
 'gran servicio celebramos cumpleanos genial',
 'deliciosa comida buen servicio limpio pacifico lugar necesita mantenimiento',
 'buen restaurante ciudad mexico restaurante elegante vecindario seguro costoso',
 'mejores lugares comer mexico temporada lluvias puedes perderte escamoles',
 'acuerdo',
 'conoci amigos hos hos b aqui aprendi mexicano cerni esada genial 10 10',
 'siempre excelente servicio comida',
 'extraordinario',
 'acuerdo',
 'lugar excelente comer hamburgoues hijo deliciosas definitiva reina hamburguesas',
 'encanta',
 'mmm delicioso',
 'super',
 'wowwwwwww magnifica experiencia aqui restaurante canada amigo trajo aqui tan agradecidos venimos platos increibles servicio realmente superior promedio quiero decir personal servidores excelentes tambien encanto comida suc

In [22]:
topic_model = BERTopic(language="multilingual")
topics, probs = topic_model.fit_transform(textos)
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,2355,-1_lugar_si_comida_mas,"[lugar, si, comida, mas, bien, comer, sabor, a...",[excelente experiencia menu definitivamente pr...
1,0,176,0_precios_precio_accesibles_surtido,"[precios, precio, accesibles, surtido, buenos,...","[rico buen precio, rico buen precio, buen precio]"
2,1,173,1_lugar_comida_yucateca_agradable,"[lugar, comida, yucateca, agradable, deliciosa...",[comida excelente lugar agradable buena atenci...
3,2,138,2_excelente_excelentes_perfecto_simplemente,"[excelente, excelentes, perfecto, simplemente,...","[excelente, excelente, excelente]"
4,3,131,3_precios_precio_comida_rica,"[precios, precio, comida, rica, altos, costos,...","[comida rica buen precio, rica comida buen pre..."
...,...,...,...,...,...
176,175,11,175_cenar_fine_tranquilamente_dining,"[cenar, fine, tranquilamente, dining, coqueto,...","[buen lugar comer cenar atencion excelente, ex..."
177,176,10,176_tacos_recibirla_paro_chiquitas,"[tacos, recibirla, paro, chiquitas, comerlo, p...",[opcion vasito consome plato birria tacos dora...
178,177,10,177_exquisita_mojarras_grandiosa_viernes,"[exquisita, mojarras, grandiosa, viernes, eleg...","[comida exquisita, exquisita comida, comida ex..."
179,178,10,178_agua_cielos_tanques_splenda,"[agua, cielos, tanques, splenda, avia, equipos...",[servicio bueno platillos excelentes creo debe...


In [36]:
aux = topic_model.get_topic_info()

new_res = {}
for t in aux['Topic'].unique():
    lista = []
    res = topic_model.get_topic(t, full=True)
    for pair in res['Main']:
        lista.append(pair[0])
    new_res[t] = lista
aux = pd.DataFrame.from_dict(new_res, orient='index')
aux.columns = ['Tema 1', 'Tema 2', 'Tema 3', 'Tema 4', 'Tema 5', 'Tema 6', 'Tema 7','Tema 8', 'Tema 9', 'Tema 10']
display(aux)
common_values = aux.stack().value_counts()
common_values.head(10)

,Tema 1,Tema 2,Tema 3,Tema 4,Tema 5,Tema 6,Tema 7,Tema 8,Tema 9,Tema 10
-1,lugar,si,comida,mas,bien,comer,sabor,atencion,solo,platillos
0,precios,precio,accesibles,surtido,buenos,sazon,razonables,rico,relacion,accesible
1,lugar,comida,yucateca,agradable,deliciosa,bonito,rica,excelente,alimentos,increible
2,excelente,excelentes,perfecto,simplemente,siemplemente,excelentisimo,zason,perfectas,atole,atendieron
3,precios,precio,comida,rica,altos,costos,accesible,caro,accesibles,razonables
...,...,...,...,...,...,...,...,...,...,...
175,cenar,fine,tranquilamente,dining,coqueto,cena,tranquila,ciento,cien,chica
176,tacos,recibirla,paro,chiquitas,comerlo,previas,deliociosas,tezozomoc,pelear,tortillinas
177,exquisita,mojarras,grandiosa,viernes,elegante,increible,espectacular,comida,rica,bonito
178,agua,cielos,tanques,splenda,avia,equipos,bombeo,refacciones,eau,purificadores


             67
comida       14
servicio     14
rico         13
excelente    12
lugar        10
buen         10
rica          9
sabor         8
atencion      8
Name: count, dtype: int64

In [25]:
topic_model.get_topic(0, full=True)

{'Main': [('precios', 0.046740180278884144),
  ('precio', 0.04530831502910763),
  ('accesibles', 0.02753516700220115),
  ('surtido', 0.021289618744527887),
  ('buenos', 0.016178638495995608),
  ('sazon', 0.013973630116078791),
  ('razonables', 0.013463356065816813),
  ('rico', 0.013017011526141268),
  ('relacion', 0.012975592176890193),
  ('accesible', 0.012776206001294284)]}

In [26]:
topic_model.visualize_topics()